Author: Nicholas Bornman

# Preamble

In [ ]:
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.display import display, clear_output
from collections import namedtuple

In [ ]:
from qick.pyro import make_proxy

from qickutils.experiment import *
from qickutils.dataset import Dataset, process_packages
Dataset.pull_packages = False

### QICK

In [ ]:
proxy_name = "board0"
soc, soccfg = make_proxy(ns_host="192.168.5.2", ns_port=8888, proxy_name=proxy_name)
print(soccfg)

In [ ]:
import scipy.fftpack
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter

# Single qubit randomised benchmarking

## Single-qubit Clifford group

The following dictionary below is the Clifford group for a single qubit. The group itself is unique, but the exact choice of group generators can vary. The convention I've chosen follows closely the table given on pages 68-69 of this thesis: https://qudev.phys.ethz.ch/static/content/science/Documents/master/Samuel_Haberthur_Masterthesis.pdf.

Each key in the "single_qubit_clifford_gates" dictionary below denotes a gate in the Clifford group, and the corresponding value is the matrix representation of that gate. The 24 gates, in practice, consist of one or more physical pulses or virtual operations (mainly in the case of Z-type gates). When the keys (i.e. gates) consist of a number of pulses (i.e. when there is a comma separating distinct pulses), the pulse key comprising the gate should be read from left to right, which corresponds with the order IN TIME in which the pulses are applied to the qubit. For example, for the "X,-Y/2" key/gate, we first send out an X pulse (namely a $\pi$ pulse in the I quadrature), followed by a -Y/2 pulse (a $-\pi/2$ pulse in the Q quadrature).

In [ ]:
single_qubit_clifford_gates = {
    "I": np.array([[-1j, 0],[0, -1j]]),
    "X": np.array([[0, -1j],[-1j, 0]]),
    "Y": np.array([[0, -1],[1, 0]]),
    "Z": np.array([[-1j, 0],[0, 1j]]),
    "X/2": (1/np.sqrt(2)) * np.array([[1, -1j],[-1j, 1]]),
    "Y/2": (1/np.sqrt(2)) * np.array([[1, -1],[1, 1]]),
    "Z/2": (1/np.sqrt(2)) * np.array([[1-1j, 0],[0, 1+1j]]),
    "-X/2": (1/np.sqrt(2)) * np.array([[1, 1j],[1j, 1]]),
    "-Y/2": (1/np.sqrt(2)) * np.array([[1, 1],[-1, 1]]),
    "-Z/2": (1/np.sqrt(2)) * np.array([[1+1j, 0],[0, 1-1j]]),
    "X/2,Y/2": (1/2) * np.array([[1+1j, -1-1j],[1-1j, 1-1j]]),
    "X/2,-Y/2": (1/2) * np.array([[1-1j, 1-1j],[-1-1j, 1+1j]]),
    "-X/2,Y/2": (1/2) * np.array([[1-1j, -1+1j],[1+1j, 1+1j]]),
    "-X/2,-Y/2": (1/2) * np.array([[1+1j, 1+1j],[-1+1j, 1-1j]]),
    "Y/2,X/2": (1/2) * np.array([[1-1j, -1-1j],[1-1j, 1+1j]]),
    "Y/2,-X/2": (1/2) * np.array([[1+1j, -1+1j],[1+1j, 1-1j]]),
    "-Y/2,X/2": (1/2) * np.array([[1+1j, 1-1j],[-1-1j, 1-1j]]),
    "-Y/2,-X/2": (1/2) * np.array([[1-1j, 1+1j],[-1+1j, 1+1j]]),
    "X,Y/2": (1/np.sqrt(2)) * np.array([[1j, -1j],[-1j, -1j]]),
    "X,-Y/2": (1/np.sqrt(2)) * np.array([[-1j, -1j],[-1j, 1j]]),
    "Y,X/2": (1/np.sqrt(2)) * np.array([[-1j, -1],[1, 1j]]),
    "Y,-X/2": (1/np.sqrt(2)) * np.array([[1j, -1],[1, -1j]]),
    "X/2,Y/2,X/2": (1/np.sqrt(2)) * np.array([[0, -1-1j],[1-1j, 0]]),
    "-X/2,Y/2,-X/2": (1/np.sqrt(2)) * np.array([[0, -1+1j],[1+1j, 0]])
    }

For convenience we create a list of the keys of the above Clifford gates

In [ ]:
clifford_group = [k for k in single_qubit_clifford_gates.keys()]

clifford_group

The "construct_cayley_table" function below constructs the Cayley table (https://en.wikipedia.org/wiki/Cayley_table) for the input group, which must be of the same form (namely a dictionary of string keys and numpy array values) as the above "single_qubit_clifford_gates". A Cayley table is essentially the lookup table for all possible products of two elements in the gateset. This function will be useful in the future when we want to move onto two-qubit gates.

In [ ]:
def construct_cayley_table(gateset):

    cayley_dict = {}

    # iterations=0

    iden = np.identity(2)

    for k_row, v_row in gateset.items():
        for k_col, v_col in gateset.items():

            # iterations += 1

            ans = v_col @ v_row

            # print(f"Current iteration: {iterations}\n")
            # print(f"Searching for {ans}\n")

            found_match = False

            matches=0

            for k_tar, v_tar in gateset.items():

                # For the matching group element v_tar which corresponds to ans,
                # 'mat' below should equal the identity, up to some phase
                mat = ans.conj().T @ v_tar

                # mat with global phase removed
                mat = np.exp(-1j* np.angle(mat[0,0])) * mat

                if np.allclose(mat, iden, atol=1e-06):
                    cayley_dict[(k_row, k_col)] = k_tar
                    found_match = True
                    matches += 1

            # print(f"Found match for iteration {iterations}? {found_match}")
            
            if not found_match or matches != 1:
                raise Exception(f"Didn't find a match for product ans: {ans},\nwhich this scheme really should have\nor found too many matches")
    
    # check integrity of Cayley table

    gates = [k for k in gateset.keys()]
    gates.sort()

    for gate in gates:

        list_temp = []

        for k, v in cayley_dict.items():
            if k[0] == gate:
                list_temp.append(v)
        
        list_temp.sort()

        if list_temp != gates:
            raise Exception(f"The Cayley tale row corresponding to {gate} does not contain every element of the Clifford group")

    return cayley_dict

Construct the Cayley table for the single qubit Clifford group.

In [ ]:
single_qubit_cayley_table = construct_cayley_table(single_qubit_clifford_gates)

Use the Cayley table to compute the `inverse' dictionary of the input Clifford gateset: the key of the dictionary is a Clifford group element, whereas the value is the inverse group element.

In [ ]:
def compute_clifford_inverses(cayley_table, input_gateset):

    inverses = {}

    for gate in input_gateset:

        for k, v in cayley_table.items():
            if k[0] == gate and v == "I":
                inverses[gate] = k[1]
                break
    
    return inverses

In [ ]:
clifford_set_inverses = compute_clifford_inverses(single_qubit_cayley_table, clifford_group)

clifford_set_inverses

## Random Clifford sequence

We'll need to construct random sequences of $n$ Clifford gates (so the sequence length is obviously $n$). That's what the following does.

Every time you call the following function, it should output a new random sequence of $n$ gates.

In [ ]:
import random

random.seed(1234) # choose an initial starting seed - no need to change this number

In [ ]:
n = 5

random.choices(clifford_group, k=n)

## Single qubit pulse library

I've assumed that all qubit drive pulse waveforms will be Gaussians, and that we'll scale the waveform's gain and length and change its phase in order to create the pulses that'll be used to construct the Clifford group elements.

In [ ]:
def gauss(mu, sigma, length, maxv=32_000):

    x = np.arange(0, length)
    y = 1/np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-mu)**2/2/sigma**2)
    y = y-y[0]
    y = (y/np.max(y))*maxv

    return y

def gaussian_pulse(sigma, mu=None):

    if mu == None:
        mu = 2*sigma

    return gauss(16*mu, 16*sigma, 2*16*mu)

In [ ]:
# sigma_temporary= # this is the sigm
# mu_temporary=2*sigma_temporary

# plt.plot(gaussian_pulse(sigma_temporary, mu_temporary))

The "create_pulse_dictionary" method below outputs the dictionary of fundamental single qubit pulses used to construct our Clifford group. It takes self as an argument, since this method will be called within the forthcoming SingleQubitRB class itself.

For the $Z$ and $\pm Z/2$ pulses, we simply change the reference phase of all subsequent carrier tones, whereas for the $X$, $Y$, $\pm X/2$, $\pm Y/2$ pulses, we simply set the Guassian waveform in either the I or Q quadrature (with a potential minus sign). For the $I$ "pulse" (i.e. doing nothing), we just wait the length of a single $\pi$ pulse (by setting the gain of an otherwise $X$ pulse to 0).

Note that we could also create the $Z$-type gates using combinations of $X$-type and $Y$-type pulses, and remove the $Z$-type entries from "create_pulse_dictionary". But for this exercise, we simply implement the $Z$-type pulses virtually.

NOTE: would it be interesting to try $X$-type and $Y$-type pulses for the $Z$-type gates?

In [ ]:
def create_pulse_dictionary(self):

    waveform = gaussian_pulse(self.sigma, self.mu)

    pulses = {
        "I": {
            "idata": waveform,
            "qdata": 0*waveform,
            "phase": 0,
            "gain": 0,
            "length": self.pi_length
            },
        "X": {
            "idata": waveform,
            "qdata": 0*waveform,
            "phase": 0,
            "gain": self.pi_gain,
            "length": self.pi_length
            },
        "Y": {
            "idata": 0*waveform,
            "qdata": waveform,
            "phase": 0,
            "gain": self.pi_gain,
            "length": self.pi_length
            },
        "Z": {
            "phase_offset": 180
        },
        "X/2": {
            "idata": waveform,
            "qdata": 0*waveform,
            "phase": 0,
            "gain": self.pi_over_2_gain,
            "length": self.pi_over_2_length
            },
        "Y/2": {
            "idata": 0*waveform,
            "qdata": waveform,
            "phase": 0,
            "gain": self.pi_over_2_gain,
            "length": self.pi_over_2_length
            },
        "Z/2": {
            "phase_offset": 90
            },
        "-X/2": {
            "idata": -1*waveform,
            "qdata": 0*waveform,
            "phase": 0,
            "gain": self.pi_over_2_gain,
            "length": self.pi_over_2_length
            },
        "-Y/2": {
            "idata": 0*waveform,
            "qdata": -1*waveform,
            "phase": 0,
            "gain": self.pi_over_2_gain,
            "length": self.pi_over_2_length
            },
        "-Z/2": {
            "phase_offset": -90
        }
    }

    return pulses

## Single qubit RB QICK class

This section contains the SingleQubitRB class, which, for a sequence of Clifford gates, plays the qubit pulses corresponding to each Clifford gate (in the order specified by the sequence), plays the pulse corresponding to the cumulative gate's inverse (using the "clifford_set_inverses" lookup), and then pulses the resonator and triggers the readout.

For a single sequence, this is repeated "repetitions" number of times.

NB: In order to run the experiment, the following $\pi$ and $\pi/2$ pulse lengths and gains need to have been calibrated already. So you need to set the folllowing class attributes:

* "pi_gain"
* "pi_over_2_gain"
* "pi_length"
* "pi_over_2_length"
* "mu" # you may want to try setting mu to be twice sigma - that's a convention I've seen being used
* "sigma"

"mu" and "sigma" correspond with the Gaussian waveform input parameters.

You need to set/calibrate the above things, in addition to the usual things during tune-up (such as the resonator and qubit frequencies, etc.)

Disclaimer: I have NOT had a chance to test the SingleQubitRB class at all, and there are bound to be bugs hiding in it.

In [ ]:
@ExperimentFactory
class SingleQubitRB(ParametrizedProgram):

    signal = Parameter('signal', type=DataParameter, source='readout_channel', trace='readout_window_length', units='', process_fnc='process_signal')

    # Readout section
    readout_channel = Parameter('Readout resonator channel')
    readout_window_length = Parameter('Readout window length', default=2, units='us')
    readout_trig_offset = Parameter('Readout ADC trigger offset', default=200, units='ns')
    readout_demodulation_detuning_frequency = Parameter('Readout demodulation detuning frequency', units='MHz', default=0)

    # Resonator drive section
    resonator_drive_channel = Parameter('Resonator drive channel')
    resonator_drive_nyquist_zone = Parameter('Resonator drive Nyquist zone', default = 2)

    resonator_frequency = Parameter('Resonator frequency', units='MHz', default=5000)

    resonator_drive_pulse_amplitude = Parameter('Resonator drive pulse amplitude', units='a.u.', default = 10_000)
    resonator_drive_pulse_phase = Parameter('Resonator drive pulse phase', units='degrees', default=0)
    resonator_drive_pulse_duration = Parameter('Resonator drive pulse duration', units='us', default=1)
    #resonator_drive_pulse_style = Parameter("Resonator pulse style")

    # Qubit section
    qubit_drive_channel = Parameter('Qubit drive channel')
    qubit_drive_nyquist_zone = Parameter('Qubit drive Nyquist zone', default = 1)

    qubit_frequency = Parameter('Qubit frequency', units='MHz', default = 4000)

    pi_gain = Parameter('Qubit pi pulse amplitude/gain', units='a.u.', default = 3000)
    #qubit_drive_pulse_phase = Parameter('Qubit drive pulse phase', units='degrees', default=0)
    pi_length = Parameter('Qubit pi pulse duration', units='ns', default=50)

    pi_over_2_gain = Parameter('Qubit pi/2 pulse amplitude/gain', units='a.u.', default = 1500)
    #qubit_drive_pulse_phase = Parameter('Qubit drive pulse phase', units='degrees', default=0)
    pi_over_2_length = Parameter('Qubit pi/2 pulse duration', units='ns', default=50)

    mu = Parameter('Gaussian waveform mean', units='clock cycles', default = 0)
    sigma = Parameter('Gaussian waveform standard deviation', units='clock cycles', default = 100)
    
    qubit_resonator_pulse_delay = Parameter('Qubit pulse & readout resonator pulse delay', units='us', default = 0.05)

    relax_delay_between_shots = Parameter('Relax delay between shots', units='us', default=500) #this should be roughly 5 times the qubit T1

    single_gate_sequence = Parameter('List of strings specifying a gate sequence', default=None)

    repetitions = Parameter('Repetitions', default=2000)


    def __init__(self, soc = None, soccfg = None, defaults=None):
        super().__init__(soc, soccfg, config=defaults)


    def process_signal(self, data, channels=None, args=None):
        return data[0, ..., 0]

    # this plays a single qubit pulse, corresponding to one of the
    # dictionary entries in the self.pulse_set object (which in turn
    # comes from the create_pulse_dictionary method from earlier)
    def play_pulse(self, pulse):

        # get information corresponding to the pulse to be played
        pulse_info = self.pulse_set[pulse]

        # if Z-type pulse, simply advanced the carrier's phase
        if pulse in ["Z", "Z/2", "-Z/2"]:
            self.qubit_drive_reference_phase += pulse_info["phase_offset"]

        # otherwise, set the register settings appropriate for the
        # pulse, and play the pulse
        else:
            self.set_pulse_registers(
                ch=self.qubit_drive_channel,
                freq=self.freq2reg(
                    self.qubit_frequency,
                    gen_ch=self.qubit_drive_channel,
                    ro_ch=self.readout_channel
                    ),
                phase=self.deg2reg(
                    self.qubit_drive_reference_phase + pulse_info["phase"],
                    gen_ch=self.qubit_drive_channel
                    ), # phase is the sum of the global reference phase and the phase of the pulse from the dictionary
                gain=int(pulse_info["gain"]), # gain of the pulse: either 0, or pi_gain or pi_over_2_gain
                waveform=pulse, # the name of the pulse waveform. NOTE: I'm assuming that we can pre-load the waveforms in 
                                # the QICK registers using the 'add_pulse' method in the initialize section below
                phrst=0,
                length=self.us2cycles(
                    pulse_info["length"],
                    gen_ch=self.qubit_drive_channel
                    ), # pulse length either corresponding to pi_length or pi_over_2_length
                mode="oneshot" #NOTE: 'oneshot' seems correct, although I'm not 100% sure what mode to use here
            ) # NOTE: do we need a "style" argument in the set_pulse_registers method above. I don't think so but I'm not sure

            # actually play the pulse using the register settings we just set

            self.pulse(ch=self.qubit_drive_channel)

    # Play the sequence of pulses corresponding to the self.single_gate_sequence
    # list, and finally play the pulse corresponding to the inverse gate
    def play_sequence(self):

        self.current_state = "I"

        for clifford_gate in self.single_gate_sequence:
            for pulse in clifford_gate.split(","):

                self.play_pulse(pulse)

                # NOTE: I'm not sure if we need to call some sort
                # of 'wait' or 'sync' method in order to wait until
                # the above pulse is finished playing before the next
                # one is played, or else if we should implement some sort
                # of delay between pulses which comprise the Clifford gates.
                # We will definitely need to add a modification here
                # when we try two-qubit benchmarking, in order to sync
                # simultaneously pulses between different qubits
            
            # update the theoretical unitary so far
            current_state = single_qubit_cayley_table[(current_state, clifford_gate)]

        # compute the inverse gate, and then play it
        inverse_gate = clifford_set_inverses[current_state]
        self.play_pulse(inverse_gate)

    
    def initialize(self):

        # set reference phase to start at 0 - this is for any virtual Z pulses
        self.qubit_drive_reference_phase = 0

        # get the pulse dictionary
        self.pulse_set = create_pulse_dictionary()


        # RESONATOR DRIVE SECTION #
        self.declare_gen(ch = self.resonator_drive_channel, nqz = self.resonator_drive_nyquist_zone)

        res_freq = self.freq2reg(self.resonator_frequency, gen_ch = self.resonator_drive_channel, ro_ch = self.readout_channel)
        res_amp = int(self.resonator_drive_pulse_amplitude)
        res_phase = self.deg2reg(self.resonator_drive_pulse_phase, gen_ch=self.resonator_drive_channel)
        res_length = self.us2cycles(self.resonator_drive_pulse_duration, gen_ch=self.resonator_drive_channel)

        # res_pulse_style = self.resonator_drive_pulse_style
        # NOTE: Do we want to uncomment the above resonator drive pulse style, and use
        # it in the set_pulse_register call below? Right now the pulse style is 'const',
        # which corresponds to a constant square waveform.
        # If we change the res_pulse_style to a Gaussian, it could technically be a different waveform
        # from the Gaussian used to construct the qubit pulse library (different in terms of sigma and mu)
                        
        self.set_pulse_registers(
            ch = self.resonator_drive_channel,
            style = 'const', # TODO change the pulse style?
            length = res_length,
            freq = res_freq,
            phase = res_phase,
            gain = res_amp
        )


        # RESONATOR READOUT SECTION #
        demod_freq = self.resonator_frequency + self.readout_demodulation_detuning_frequency
        readout_length = self.us2cycles(self.readout_window_length, ro_ch = self.readout_channel)

        self.declare_readout(ch=self.readout_channel, length=readout_length, freq=demod_freq, gen_ch = self.resonator_drive_channel)


        # QUBIT SECTION #

        self.declare_gen(ch = self.qubit_drive_channel, nqz = self.qubit_drive_nyquist_zone)
        self.default_pulse_registers(ch=self.qubit_drive_channel, style="arb")
        # NOTE: do we need this? I'm not sure that we do, given that we're specifying
        # the I and Q for each pulse

        # QUBIT PULSE LIBRARY SECTION #
        # load the dictionary of pulses that will be played by the
        # "play_pulse" method - it's specified in that method via the
        # "waveform" argument in self.set_pulse_registers
        # NOTE: I'm hoping that we can load all of the I and Q waveforms
        # in memory and simply change their gains, phases, etc., on
        # the fly in the play_pulse method, but I'm not totally sure

        for k, v in self.pulse_set.items():

            self.add_pulse(
                ch=self.qubit_drive_channel,
                name=k,
                idata=v["idata"],
                qdata=v["qdata"],
            )
        
        # GATE SEQUENCE CHECK #
        # the random gate sequence we wish to play, cannot be None

        if self.single_gate_sequence == None:
            raise TypeError("You need to define a gate sequence.")

        self.sync_all(self.us2cycles(100))


    def body(self):

        # play the sequence of pulses defined by the single_gate_sequence parameter
        self.play_sequence()
        
        # set a delay between playing the last pulse in the sequence of qubit pulses, and
        # pulsing the resonator
        self.sync_all(
            self.us2cycles(
                self.qubit_resonator_pulse_delay,
                gen_ch=self.qubit_drive_channel
            )
        )
        
        # pulse the readout resonator and trigger the adc to read out the
        # resonator state
        self.pulse(ch=self.resonator_drive_channel)
        self.trigger(
            adcs=[self.readout_channel],
            adc_trig_offset=self.us2cycles(self.readout_trig_offset)
        )

        # after triggering the adc data collection, call sync_all to wait
        # a certain amount of time before the next iteration of the
        # enclosing 'body' method
                
        self.wait_all()
        self.sync_all(self.us2cycles(self.relax_delay_between_shots))

# Run single-qubit RB measurement

In [ ]:
prg = SingleQubitRB(soc=soc, soccfg=soccfg)
prg.debug_traces = False

Set some of the usual class attributes that you're used to.

In [ ]:
prg.readout_channel = 0
prg.readout_window_length = 1
prg.readout_trig_offset = 200
prg.readout_demodulation_detuning_frequency = 0

prg.resonator_drive_channel = 1
prg.resonator_drive_nyquist_zone = 2

prg.resonator_frequency = 5_000

prg.resonator_drive_pulse_amplitude = 10_000
prg.resonator_drive_pulse_phase = 0
prg.resonator_drive_pulse_duration = 1

prg.qubit_drive_channel = 2
prg.qubit_drive_nyquist_zone = 1

prg.qubit_frequency = 4_000

prg.sigma = soc.us2cycles(0.01, gen_ch=prg.qubit_drive_channel) # NOTE: check that the getter of prg actually returns the value - I'm not sure how the getter for the prg object is set up
prg.mu = 2*prg.sigma

prg.qubit_resonator_pulse_delay = 0.05

prg.relax_delay_between_shots = 500 # this should be bigger than 5 times the expected/measured qubit T1

prg.repetitions = 2000 # for every unique clifford sequence, take this many IQ single shot measurements - we do not want to average the 2000 shots yet

Calibrate the following parameters before running the measurement:

* pi_gain is the gain, in arbitrary DAC units, of the $\pi$ pulses that are played in the "play_pulse" method of the SingleQubitRB class
* pi_length is the time length, in $\mu s$, of the $\pi$ pulses that are played in the "play_pulse" method of the SingleQubitRB class
* pi_over_2_gain is the gain, in arbitrary DAC units, of the $\pi/2$ pulses that are played in the "play_pulse" method of the SingleQubitRB class
* pi_over_2_length is the time length, in $\mu s$, of the $\pi/2$ pulses that are played in the "play_pulse" method of the SingleQubitRB class

In [ ]:
prg.pi_gain = 3000
prg.pi_length = 0.05

prg.pi_over_2_gain = 1500
prg.pi_over_2_length = 50

## One random Clifford gate sequence

Update the single_gate_sequence to a new one of length 5. This subsection is mainly for you to test whether everything appears to be working.

In [ ]:
prg.single_gate_sequence = random.choices(clifford_group, k=5)

NB: You should save the single_gate_sequence (perhaps in a csv file or pandas dataframe - in the next subsection I'll use a pickle dictionary for simplicity)

The following actually executes the experment - you may need to change some of the .execute parameters.

In [ ]:
ds = prg.execute(f'<SPECIFY DIRECTORY WHERE DATA WILL BE SAVED, HERE>', suffix = 'test', grid = (), show_progress = show_progress)

Once the experiment is complete, you would access the data by indexing the ds object above - I haven't had time to test how to access the data inside ds itself and parse it, but you guys should be able to figure this out yourselves.

In [ ]:
# data, axes, labels, legends = [None], [None], [None], [None]

# data[i], axes[i], labels[i], legends[i] = ds[-1].take(f"signal")

data, axes, labels, legends = ds.take(f"signal")

## Full experiment - multiple repetitions over multiple random sequences, over many different sequence lengths

The following should run the actual RB experiment in full

In [ ]:
import csv
 
fields = ['Sequence index', 'Sequence']

for n in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]: # Clifford sequence lengths

    with open(f"/Users/nbornman-local/Desktop/clifford_sequence_length_{n}.csv", "w") as sequence_file:

        sequence_writer = csv.writer(sequence_file)

        sequence_writer.writerow(fields)

        for i in range(0, 1_000): # the number of sequences of length "n" we'll run

            sequence = random.choices(clifford_group, k=n)

            prg.single_gate_sequence = sequence

            ds = prg.execute(f'single_qubit_RB_data_length_{n}_sequence_{i}', suffix = '', grid = (), show_progress = show_progress)

            sequence_writer.writerow([i, sequence])
        
        print("Completed experiments for length {n}\n")

That's it!